# Содержание #

<a id='import_libs'></a>
# Импорт библиотек #

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn
import warnings
import numpy as np
import scipy.stats as st
from IPython.core.display import display

warnings.filterwarnings('ignore')

<a id='data'></a>
# Данные #

In [2]:
data = pd.read_csv(filepath_or_buffer='datasets/games.csv')

<a id='data_view'></a>
## Первичный осмотр данных ##

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16715 entries, 0 to 16714
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Name             16713 non-null  object 
 1   Platform         16715 non-null  object 
 2   Year_of_Release  16446 non-null  float64
 3   Genre            16713 non-null  object 
 4   NA_sales         16715 non-null  float64
 5   EU_sales         16715 non-null  float64
 6   JP_sales         16715 non-null  float64
 7   Other_sales      16715 non-null  float64
 8   Critic_Score     8137 non-null   float64
 9   User_Score       10014 non-null  object 
 10  Rating           9949 non-null   object 
dtypes: float64(6), object(5)
memory usage: 1.4+ MB


In [4]:
data

,Name,Platform,Year_of_Release,Genre,NA_sales,EU_sales,JP_sales,Other_sales,Critic_Score,User_Score,Rating
0,Wii Sports,Wii,2006.0,Sports,41.36,28.96,3.77,8.45,76.0,8,E
1,Super Mario Bros.,NES,1985.0,Platform,29.08,3.58,6.81,0.77,NaN,NaN,NaN
2,Mario Kart Wii,Wii,2008.0,Racing,15.68,12.76,3.79,3.29,82.0,8.3,E
3,Wii Sports Resort,Wii,2009.0,Sports,15.61,10.93,3.28,2.95,80.0,8,E
4,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,11.27,8.89,10.22,1.00,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
16710,Samurai Warriors: Sanada Maru,PS3,2016.0,Action,0.00,0.00,0.01,0.00,NaN,NaN,NaN
16711,LMA Manager 2007,X360,2006.0,Sports,0.00,0.01,0.00,0.00,NaN,NaN,NaN
16712,Haitaka no Psychedelica,PSV,2016.0,Adventure,0.00,0.00,0.01,0.00,NaN,NaN,NaN
16713,Spirits & Spells,GBA,2003.0,Platform,0.01,0.00,0.00,0.00,NaN,NaN,NaN


# Результаты первичного осмотра данных #
1. перевести название столбцов в нижний регистр
2. перевести строковые значения в нижний регистр, удалить лишние символы
3. data['Name'] удалить строки содержащие NaN
4. data['Plaform'] привести к типу 'category'
5. data['Year_of_Release']
    * обогатить на основании названия игры
    * заменить пропуски на 'undefined'
6. data['Genre']
    * привести к типу 'category'
7. data['Critic_Score']
    * заменить пропущенные значения на 0
8. data['User_Score']
    * заменить пропущенные значения на 0
    * привести к типу int
    * привести значения к масштабу data['Critic_score']
9. data['Rating'] привести к типу 'category'

<a id='prepare'></a>
# Предобработка данных #

<a id='prepare_columns'></a>
## Переименование столбцов ##

In [5]:
data.columns = data.columns.str.lower()
data.columns = data.columns.str.strip()
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16715 entries, 0 to 16714
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   name             16713 non-null  object 
 1   platform         16715 non-null  object 
 2   year_of_release  16446 non-null  float64
 3   genre            16713 non-null  object 
 4   na_sales         16715 non-null  float64
 5   eu_sales         16715 non-null  float64
 6   jp_sales         16715 non-null  float64
 7   other_sales      16715 non-null  float64
 8   critic_score     8137 non-null   float64
 9   user_score       10014 non-null  object 
 10  rating           9949 non-null   object 
dtypes: float64(6), object(5)
memory usage: 1.4+ MB


<a id='prepare_string'></a>
## Обработка строковых значений ##

In [6]:
def prepare_string():
    for column in data.columns:
        if data.dtypes[column] == 'object':
            data[column] = data[column].str.lower()
            data[column] = data[column].str.strip()

prepare_string()

## Удаление строк с пропусками data['name']

In [7]:
data.dropna(subset=['name'], inplace=True)
data.isna().sum()

name                  0
platform              0
year_of_release     269
genre                 0
na_sales              0
eu_sales              0
jp_sales              0
other_sales           0
critic_score       8576
user_score         6699
rating             6764
dtype: int64

<a id='prepare_year_of_release'></a>
## Обогащение data['year_of_release'] ##

In [8]:
year_range = range(data['year_of_release'].min().astype('int'), data['year_of_release'].max().astype('int'))

def year_update(raw):
    if np.isnan(raw['year_of_release']):
        for word in raw['name'].split(' '):
            if word.isnumeric() and pd.to_numeric(word) in year_range:
                return pd.to_numeric(word) - 1
    else: return pd.to_numeric(raw['year_of_release'])

data['year_of_release'] = data.apply(year_update, axis=1)
data['year_of_release'] = data['year_of_release'].fillna(np.NaN)
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16713 entries, 0 to 16714
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   name             16713 non-null  object 
 1   platform         16713 non-null  object 
 2   year_of_release  16459 non-null  float64
 3   genre            16713 non-null  object 
 4   na_sales         16713 non-null  float64
 5   eu_sales         16713 non-null  float64
 6   jp_sales         16713 non-null  float64
 7   other_sales      16713 non-null  float64
 8   critic_score     8137 non-null   float64
 9   user_score       10014 non-null  object 
 10  rating           9949 non-null   object 
dtypes: float64(6), object(5)
memory usage: 1.5+ MB


<a id='prepare_tbd_user_score'></a>
## Замена значения "tbd" data['user_score'] ##
Пояснение: данные на основе которых можно заменить значение "tbd" отсутствуют => проведем замену на Nan

In [9]:
data['user_score'] = data['user_score'].replace(to_replace='tbd',
                                                value=np.NaN)

<a id='prepare_user_score'></a>
## Изменение типа данных data['user_score'], масштабирование данных ##

In [10]:
data['user_score'] = pd.to_numeric(data['user_score'], errors='coerce') * 10
print(data.info())
print(data['user_score'].unique())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16713 entries, 0 to 16714
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   name             16713 non-null  object 
 1   platform         16713 non-null  object 
 2   year_of_release  16459 non-null  float64
 3   genre            16713 non-null  object 
 4   na_sales         16713 non-null  float64
 5   eu_sales         16713 non-null  float64
 6   jp_sales         16713 non-null  float64
 7   other_sales      16713 non-null  float64
 8   critic_score     8137 non-null   float64
 9   user_score       7590 non-null   float64
 10  rating           9949 non-null   object 
dtypes: float64(7), object(4)
memory usage: 1.5+ MB
None
[80. nan 83. 85. 66. 84. 86. 77. 63. 74. 82. 90. 79. 81. 87. 71. 34. 53.
 48. 32. 89. 64. 78. 75. 26. 72. 92. 70. 73. 43. 76. 57. 50. 91. 65. 88.
 69. 94. 68. 61. 67. 54. 40. 49. 45. 93. 62. 42. 60. 37. 41. 58. 56. 55.
 44. 46. 59. 39. 

## Проверка гипотезы об идентичности data['user_score'], data['critic_score'] ##
$H_0:$ рейтинг критиков совпадает с рейтингом пользователей

$H_1:$ рейтинг критиков не совпадает с рейтингом пользователей

In [11]:
critics_score = data.query('~critic_score.isna()')['critic_score']
users_score = data.query('~user_score.isna()')['user_score']
print(critics_score.shape[0])
print(users_score.shape[0])

8137
7590


In [12]:
alpha = .05
result = st.ttest_ind(critics_score,
                      users_score,
                      equal_var=False)
print(f'Pvalue: {result.pvalue}')
if result.pvalue < alpha:
    print("Отвергаем нулевую гипотезу")
else:
    print("Не получилось отвергнуть нулевую гипотезу")

Pvalue: 6.731311703095746e-23
Отвергаем нулевую гипотезу


<a id='prepare_change_type'></a>
## Изменение типов данных ##

In [13]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16713 entries, 0 to 16714
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   name             16713 non-null  object 
 1   platform         16713 non-null  object 
 2   year_of_release  16459 non-null  float64
 3   genre            16713 non-null  object 
 4   na_sales         16713 non-null  float64
 5   eu_sales         16713 non-null  float64
 6   jp_sales         16713 non-null  float64
 7   other_sales      16713 non-null  float64
 8   critic_score     8137 non-null   float64
 9   user_score       7590 non-null   float64
 10  rating           9949 non-null   object 
dtypes: float64(7), object(4)
memory usage: 1.5+ MB


In [14]:
data['platform'] = data['platform'].astype('category')
data['genre'] = data['genre'].astype('category')
data['rating'] = data['rating'].astype('category')
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16713 entries, 0 to 16714
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype   
---  ------           --------------  -----   
 0   name             16713 non-null  object  
 1   platform         16713 non-null  category
 2   year_of_release  16459 non-null  float64 
 3   genre            16713 non-null  category
 4   na_sales         16713 non-null  float64 
 5   eu_sales         16713 non-null  float64 
 6   jp_sales         16713 non-null  float64 
 7   other_sales      16713 non-null  float64 
 8   critic_score     8137 non-null   float64 
 9   user_score       7590 non-null   float64 
 10  rating           9949 non-null   category
dtypes: category(3), float64(7), object(1)
memory usage: 1.2+ MB
